# 問題
厚さ$d$，透磁率$\mu_0$の不均質誘電体スラブにパルス平面波が入射する場合を考える．

![](img/1D.png)

# 環境構築

In [ ]:
%pip install numpy
import numpy as np

%pip install matplotlib
import matplotlib.pyplot as plt

%pip install tqdm
from tqdm import tqdm 

%mkdir result
%rm result/*.png

# 各種パラメータの設定

In [ ]:
# 解析のパラメータ
nz    = 1000 # 解析領域の分割数
nstep = 2000 # 計算のステップ数

# 真空のパラメータ
c    = 2.9979246e8   # 光速
eps0 = 8.8541878e-12 # 真空の誘電率
mu0  = 1.2566371e-6  # 真空の透磁率
z0   = 376.73031     # 真空の特性インピーダンス

# 誘電体のパラメータ
epsr = 3.0   # 誘電体の比誘電率
d    = 0.1   # 誘電体厚さ[m]
nd   = 50    # 誘電体の分割数
kd   = nz/2  # 誘電体の左端の位置

# 解析のパラメータ
dz = d/nd # セルサイズ
dt = dz/c  # 時間ステップ

# 初期化

In [ ]:
# 誘電率, 伝導率の初期化
epsd = np.empty(nz) # 比誘電率
sgmd = np.empty(nz) # 伝導率

for k in range(nz):
    if k < kd or k >= kd+nd: # 真空中
        epsd[k] = 1.0
        sgmd[k] = 0.0
    else: # 誘電体中
        epsd[k] = epsr
        sgmd[k] = 0.0

# 電界，磁界の係数
ae = np.empty(nz)
be = np.empty(nz)

for k in range(nz):
    esp = eps0*(epsd[k] + epsd[k-1])/2
    sgm = (sgmd[k] + sgmd[k-1])/2
    b = dt / esp
    a0 = sgm * b / 2

    ae[k] = (1 - a0)/(1 + a0)
    be[k] = b/(1 + a0)/dz

bm = dt / mu0 / dz

入射波形の関数

In [ ]:
def input_pulse(z, t):
    zp = 100 * dz
    a  =  10 * dz
    return np.exp(-((z - zp - c*t)/a)**2)

初期化の続き

In [ ]:
# 電界，磁界の初期化
ex = np.empty(nz)
hy = np.empty(nz)

ex_t = np.zeros(nstep)  # 電界の時間応答
ex_t2 = np.zeros(nstep) # 電界の時間応答

for k in range(nz):
    z = k*dz
    ex[k] = input_pulse(z, 0)

    z += dz/2
    hy[k] = input_pulse(z, dt/2)/z0

# 吸収境界上の電界
exlold = ex[1]
exrold = ex[nz-1]

# Murの係数
v = c/np.sqrt(epsd[0])
czl = (v*dt - dz)/(v*dt + dz) # 左端

v = c/np.sqrt(epsd[nz-1])
czr = (v*dt - dz)/(v*dt + dz) # 右端

電磁界分布の可視化

In [ ]:
def plot(t, show=False):
    plt.figure(figsize=(6,4))
    plt.subplot(211)
    z = np.linspace(0, nz*dz, nz)
    plt.plot(z, ex)
    plt.plot([dz*kd,dz*kd]    , [-1.2,1.2], color="orange")
    plt.plot([dz*kd+d,dz*kd+d], [-1.2,1.2], color="orange")
    plt.ylabel(r'$E_z$')
    plt.xlim(0, dz*nz)
    plt.ylim(-0.4, 1.2)

    plt.subplot(212)
    z = np.linspace(dz/2, nz*dz, nz)
    plt.plot(z, hy)
    plt.plot([dz*kd,dz*kd]    , [-0.02,0.02], color="orange")
    plt.plot([dz*kd+d,dz*kd+d], [-0.02,0.02], color="orange")
    plt.ylabel(r'$H_y$')
    plt.xlim(0, dz*nz)
    plt.ylim(-0.002, 0.004)

    plt.savefig("result/{}.png".format(t))
    if show:
        plt.show()
    plt.close()

plot(1, True)

# 電磁界の計算 と Murの一次吸収条件

In [ ]:
def e_cal():
    for k in range(1, nz-1):
        ex[k] = ae[k]*ex[k] - be[k]*(hy[k] - hy[k-1])

def h_cal():
    for k in range(0, nz-1):
        hy[k] = hy[k] - bm*(ex[k+1] - ex[k])

def mur():
    global exlold
    global exrold
    
    ex[0]  = exlold + czl*(ex[1]      - ex[0] ) # 左端
    ex[nz-1] = exrold + czr*(ex[nz-2] - ex[nz-1]) # 右端

    exlold = ex[1]
    exrold = ex[nz-2]

# FDTDの計算

In [ ]:
# FDTDの計算
t = dt
for n in tqdm(range(1, nstep)):
    e_cal()
    mur()
    t += dt/2

    h_cal()
    t += dt/2
    
    ex_t[n]  = ex[350]
    ex_t2[n] = ex[800]

    if n%5 == 0:
        plot(n)


# 計算結果
電磁界分布のアニメーション

In [ ]:
%pip install Pillow
%pip install natsort

from PIL import Image
import glob
from natsort import natsorted


files = natsorted(sorted(glob.glob('result/*.png')))

images = list(map(lambda file: Image.open(file), files))

images[0].save('img/1D.gif', save_all=True, optimize=False, append_images=images[1:], duration=1, loop=0)


電界の時間応答

In [ ]:
t = np.linspace(0, nstep*dt * 1e9, nstep)

plt.plot(t, ex_t , label=r"z = {:.4g} m".format(350*dz))
plt.plot(t, ex_t2, label=r"z = {:.4g} m".format(800*dz))
plt.xlim(0, 7)
plt.xlabel("t [ns]")
plt.ylabel(r"$E_x$")
plt.legend()

plt.savefig("img/1D-time-domain.png")
plt.show()

t=2.5 ns 前後で，入射波と反射波に分かれる

フーリエ変換により透過係数と反射係数の周波数依存を計算

In [ ]:
ex_in = ex_t[:450]
ex_rf = ex_t[450:900]
ex_tr = ex_t2[550:1000]

# フーリエ変換
N = 900

F = np.fft.fft(ex_in)
freq = np.fft.fftfreq(N, d=dt/2) / 1e9
amp1 = np.abs(F/(N/2))

F = np.fft.fft(ex_rf)
freq = np.fft.fftfreq(N, d=dt/2) / 1e9
amp2 = np.abs(F/(N/2))

F = np.fft.fft(ex_tr)
freq = np.fft.fftfreq(N, d=dt/2) / 1e9
amp3 = np.abs(F/(N/2))

plt.scatter(freq[1:int(N/2)], amp2[1:int(N/2)]/amp1[1:int(N/2)])
plt.scatter(freq[1:int(N/2)], amp3[1:int(N/2)]/amp1[1:int(N/2)])

# 理論値
f = np.linspace(0.1e9, 4e9, 1000)
w = 2 * np.pi * f

k0 = w / c
k2 = w / c * np.sqrt(epsr)

z0 = 377 # 自由空間中の波動インピーダンス
y0 = 1/z0

z2 = z0/np.sqrt(epsr)
y2 = 1/z2

R = 1j * (y0*z2*y0 - y2) * np.sin(k2*d) / ((2*y0)*np.cos(k2*d) + 1j*(y0*z2*y0 + y2)*np.sin(k2*d))
T = y0 * (1-R) / np.exp(-1j*k0*d) / (1j*y2*np.sin(k2*d) + y0*np.cos(k2*d))

plt.plot(f/1e9, np.abs(R), label="Reflection")
plt.plot(f/1e9, np.abs(T), label="Transparent")

plt.xlim(0,4)
plt.ylim(0,1)
plt.xlabel("Frequency [GHz]")
plt.legend()
plt.savefig("img/1D-reflection.png")
plt.show()
